# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = os.path.join("..","WeatherPy", "Weather Check.csv")
weather_df = pd.read_csv(path)
weather_df = weather_df[['City','Temperature (F)', 'Humidity (%)', 'Cloudiness (%)', 'Wind Speed (mph)', 'Lat', 'Lon']]
weather_df

,City,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lon
0,Rikitea,75.25,63.0,96.0,22.24,NaN,NaN
1,Albany,49.66,55.0,25.0,4.45,NaN,NaN
2,Lebu,52.21,92.0,7.0,6.60,NaN,NaN
3,Bethel,53.92,66.0,20.0,3.06,NaN,NaN
4,Manturovo,61.11,90.0,98.0,5.97,NaN,NaN
...,...,...,...,...,...,...,...
1425,Yarensk,63.77,51.0,100.0,5.79,62.1675,49.0916
1426,Yuncheng,62.89,83.0,100.0,7.87,35.0231,110.9928
1427,Road Town,79.34,86.0,25.0,3.00,18.4167,-64.6167
1428,Kenora,53.29,71.0,75.0,6.91,49.8167,-94.4337


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [10]:
weather_df_complete = weather_df.dropna()
locations = weather_df_complete[["Lat", "Lon"]].astype(float)
humidity = weather_df_complete['Humidity (%)'].astype(float)

In [11]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

In [12]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
ideal_df = weather_df_complete[weather_df_complete['Temperature (F)']<80] 
ideal_df = ideal_df[ideal_df['Temperature (F)']>70]
ideal_df = ideal_df[ideal_df['Wind Speed (mph)']<10]
ideal_df = ideal_df[ideal_df['Cloudiness (%)']==0]
print(len(ideal_df))
ideal_df = ideal_df.drop_duplicates()
ideal_df = ideal_df.reset_index()
ideal_df = ideal_df[['City','Temperature (F)', 'Humidity (%)', 'Cloudiness (%)', 'Wind Speed (mph)', 'Lat', 'Lon']]
ideal_df

14


,City,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lon
0,Saint-Philippe,78.10,53.0,0.0,4.61,-21.3585,55.7679
1,Abu Samrah,70.21,36.0,0.0,8.19,35.3029,37.1841
2,Oussouye,73.40,94.0,0.0,2.30,12.4850,-16.5469
3,Saint-Joseph,77.88,53.0,0.0,4.61,-21.3667,55.6167
4,Mehrān,77.00,23.0,0.0,6.91,33.1222,46.1646
5,Gunjur,71.60,88.0,0.0,1.14,13.2019,-16.7339
6,Esna,73.40,27.0,0.0,2.30,25.2934,32.5540
7,Ures,79.41,55.0,0.0,1.99,29.4333,-110.4000
8,Chapada dos Guimarães,73.40,78.0,0.0,2.30,-15.4606,-55.7497
9,Sonoita,73.02,17.0,0.0,9.15,31.8500,-112.8333


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hotel_df = pd.DataFrame({"Hotel Name": [],
                           "City": [],
                           "Country": []})
#              "Temperature (F)": [],
#              "Humidity (%)": [],
#              "Cloudiness (%)": [],
#              "Wind Speed (mph)": []})

In [52]:
for city in ideal_df['City']:
    print(ideal_df[ideal_df['City']==city]['Lat'])
    break
    target_coordinates = f" {ideal_df[ideal_df['City']==city]['Lat']}, {ideal_df[ideal_df['City']==city]['Lon']} "
    print(target_coordinates)
    break
    target_radius = 5000
    target_type = "lodging"
    keyword = 'hotel'
    # set up a parameters dictionary
    params = {"location": target_coordinates,
              "radius": target_radius, 
              "type": target_type,
              "key": g_key}
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    # convert response to json
    hotel_data = response.json()
    pprint(hotel_data)#["results"][0]["name"])
    #print(hotel_data["results"][0]["vicinity"])

# Print the json (pretty printed)
#print(json.dumps(places_data, indent=4, sort_keys=True))

0   -21.3585
Name: Lat, dtype: float64


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [ ]:
# Add marker layer ontop of heat map


# Display figure
